# Метрика Pfound

## Imports

In [43]:
import pandas as pd
import numpy as np
import numba as nb

from tqdm.auto import tqdm

In [44]:
np.random.seed(23)

## Homework

### PFound
Исходные данные - Yandex Cup 2022 Analytics
- Ссылка - https://yandex.ru/cup/analytics/analysis/ , пример A. Рассчитать pFound
- Данные - https://yadi.sk/d/guqki4UI4hFlXQ
- Формула
$$pFound@K = \sum_{i=1}^{k} pLook[i]\ pRel[i]$$

$$pLook[1] = 1$$

$$pLook[i] = pLook[i-1]\ (1 - pRel[i-1])\ (1 - pBreak)$$

$$pBreak = 0.15$$

**Задача** - написать функцию, которая принимает на вход dataframe (после join), а на выходе дает средний pFound по всем query.
- Запрещается использовать циклы for для расчет метрики (как полностью, так и ее частей).
- Усложнение, если задача показалась легкой - попробуйте обойтись без groupby (не уверен, что это возможно, но вдруг вы справитесь)


##Решение с циклом

(Измененный пример из yandexcup)


In [45]:
# считываем данные
qid_query = pd.read_csv("qid_query.tsv", sep="\t", names=["qid", "query"])
qid_url_rating = pd.read_csv("qid_url_rating.tsv", sep="\t", names=["qid", "url", "rating"])
hostid_url = pd.read_csv("hostid_url.tsv", sep="\t", names=["hostid", "url"])

# делаем join двух таблиц
qid_url_rating_hostid = pd.merge(qid_url_rating, hostid_url, on="url")
qid_url_rating_hostid.head()

,qid,url,rating,hostid
0,10387,http://batman-arkhamcity.ru/,0.00,64
1,10387,http://bigtorrents.org/publ/batman_arkham_city...,0.14,71
2,10387,http://consolelife.ru/xbox-360/6577-o-rossiysk...,0.14,101
3,10387,http://dic.academic.ru/book.nsf/3662736/Batman...,0.00,115
4,10387,http://forum.csmania.ru/viewtopic.php?t=25986,0.14,155


In [46]:
def plook(ind, rels):
  if ind == 0:
    return 1
  return plook(ind-1, rels)*(1-rels[ind-1])*(1-0.15)


def pfound(group, top_k):
  max_by_host = group.groupby("hostid")["rating"].max() # максимальный рейтинг хоста
  topk = max_by_host.sort_values(ascending=False)[:top_k] # берем топ k урлов с наивысшим рейтингом
  pfound = 0
  for ind, val in enumerate(topk):
    pfound += val*plook(ind, topk.values)
  return pfound

In [47]:
qid_pfound = qid_url_rating_hostid.groupby('qid').apply(pfound, top_k=10) # группируем по qid и вычисляем pfound
qid_pfound

qid
10387     0.497771
20860     0.655448
21070     0.497771
35618     0.437794
107538    0.354808
150126    0.366109
168170    0.481255
176370    0.393661
192007    0.191170
213932    0.347005
221830    0.497771
242953    0.497771
253476    0.497771
295761    0.900836
346214    0.263596
347852    0.618534
360100    0.470204
366042    0.309314
375608    0.497771
380923    0.429989
dtype: float64

In [48]:
qid_pfound = qid_url_rating_hostid.groupby('qid').apply(pfound, top_k=10).mean() # группируем по qid и вычисляем средний pfound по всем query
qid_pfound

0.4603173929969002

In [49]:
def calc_plook_yandex(df, topk):
  qid_pfound = qid_url_rating_hostid.groupby('qid').apply(pfound, top_k=10) # группируем по qid и вычисляем pfound
  return qid_pfound, qid_pfound.mean()

## Решение без цикла


In [50]:
def headtail(df):
    return pd.concat([df.head(), df.tail()])

####Load data

In [67]:
def load_data():
  # считываем данные
  qid_query = pd.read_csv("qid_query.tsv", sep="\t", names=["qid", "query"])
  qid_url_rating = pd.read_csv("qid_url_rating.tsv", sep="\t", names=["qid", "url", "rating"])
  hostid_url = pd.read_csv("hostid_url.tsv", sep="\t", names=["hostid", "url"])

  # делаем join двух таблиц,
  qid_url_rating_hostid = pd.merge(qid_url_rating, hostid_url, on="url")

  #Удаляем ненужные колонки
  qid_url_rating_hostid = qid_url_rating_hostid.drop(["url"], axis=1)

  Dup_Rows = qid_url_rating_hostid[qid_url_rating_hostid.duplicated()]
  qid_url_rating_hostid = qid_url_rating_hostid.drop_duplicates(keep='first')

  qid_url_rating_hostid.sort_values(by=["qid", 'rating'], ascending=[False,False], inplace=True)
  return qid_url_rating_hostid

####PFound vectorized

In [68]:
def pfound_vector(join_df, k):
  join_df = join_df.copy()
  # Сортируем df
  join_df.sort_values(by=["qid", 'rating'], ascending=[False,False], inplace=True)
  # Удаляем дубликаты взаимодейтсвия 
  Dup_Rows = join_df[join_df.duplicated()]
  join_df = join_df.drop_duplicates(keep='first')

  # Проставляем порядок вхождения
  join_df["rank"] = join_df.groupby("qid").cumcount() + 1

  # Высчитываем коэф. * (1-rel) 
  join_df['1relbetta'] = (1 - join_df["rating"])*(1-0.15)

  # С помощью cumprod получаем plook и сдвигаем его на строку вниз, чтобы получить предыдущие значения
  # nan заполняем 1 по формуле
  join_df["cumprod"] = join_df.groupby("qid")["1relbetta"].cumprod()
  join_df["plook"] = join_df.groupby("qid")["cumprod"].shift(1)
  join_df.fillna(1, inplace=True)

  # Высчитываем pfound
  join_df["pfound"] = join_df["plook"] * join_df["rating"]

  # Берем top k
  join_df["pfound@k"] = join_df["rank"] <= k
  join_df = join_df[join_df["pfound@k"]==True]

  # Высчитываем pfound для каждого qid
  qid_pfound = join_df.groupby("qid")["pfound"].sum()


  return qid_pfound, qid_pfound.mean()

## Проверка решений

In [69]:
topk = 10
qid_url_rating_hostid = load_data()
print(headtail(qid_url_rating_hostid))
p_found_yandex, mean_pfound_yandex = calc_plook_yandex(qid_url_rating_hostid, topk)

p_found, mean_pfound = pfound_vector(qid_url_rating_hostid, topk)

print("Yandex pfound group")
print(p_found_yandex)
print("New pfound group")
print(p_found, "\n\n\n")


print("Yandex pfound mean")
print(mean_pfound_yandex)
print("New pfound mean")
print(mean_pfound)


        qid  rating  hostid
813  380923    0.14       6
821  380923    0.14     179
831  380923    0.14     386
836  380923    0.14     484
835  380923    0.07     462
38    10387    0.00     846
41    10387    0.00     922
45    10387    0.00     995
46    10387    0.00    1042
48    10387    0.00    1155
Yandex pfound group
qid
10387     0.497771
20860     0.655448
21070     0.497771
35618     0.437794
107538    0.354808
150126    0.366109
168170    0.481255
176370    0.393661
192007    0.191170
213932    0.347005
221830    0.497771
242953    0.497771
253476    0.497771
295761    0.900836
346214    0.263596
347852    0.618534
360100    0.470204
366042    0.309314
375608    0.497771
380923    0.429989
dtype: float64
New pfound group
qid
10387     0.497771
20860     0.655448
21070     0.497771
35618     0.443964
107538    0.354808
150126    0.366109
168170    0.481255
176370    0.393661
192007    0.191170
213932    0.347005
221830    0.497771
242953    0.497771
253476    0.497771
29576